convert qupath annotation to imagescope xml in order to train deeplab model in MATLAB
210802 last edit

In [25]:
from paquo.projects import QuPathProject
import numpy as np
import os
import xml.etree.ElementTree as ET
# from xml.dom import minidom
# import cv2
# from matplotlib import pyplot as plt
from readxml_b import *
from time import time
import pandas as pd

In [26]:
# Open Qupath Project to read annoation in later steps
s1 = time()
qppath = r"\\motherserverdw\Kyu_Sync\Research\Active\Aging_organized\svs\svs_back\qupath_v3\project.qpproj"
dstpth = r'\\10.162.80.18\Kyu_Sync\Research\Active\Aging_organized\svs\dl_annotation_210802';
## Uncomment below to update the image paths embedded in the qupath project in case the server ip changes.
# with QuPathProject(qppath, mode='r+') as qp:
#     if os.path.exists(qp.images[0].uri[7:]):
#         print('loading project')
#     else:
#         print('updating uri')
#         if os.path.exists(r"\\motherserverdw\Kyu_Sync"):
#             for image in qp.images:
#                 qp.update_image_paths(uri2uri={image.uri: image.uri.replace('kukissd','10.162.80.6')})
#         elif os.path.exists(r"\\kukissd\Kyu_Sync"):
#             for image in qp.images:
#                 qp.update_image_paths(uri2uri={image.uri: image.uri.replace('10.162.80.6','kukissd')})
#         else:
#             raise NameError('server is not accessible')
qp=QuPathProject(qppath, mode='r')
print('quproj opened in {:.2f} sec'.format(time() - s1))

quproj opened in 0.05 sec


In [27]:
# list all annotation classes
available_classes = [k.name for k in qp.path_classes]
available_classes

['None',
 'roi',
 'granulosum',
 'basale',
 'white',
 'spinosum',
 'collagen',
 'bloodvessel',
 'noise',
 'inner corneum',
 'outer corneum',
 'hairfollicle',
 'hairroot',
 'smoothmuscle',
 'oil',
 'sweat',
 'nerve',
 'fat',
 'inflammation',
 'vesselwall']

In [28]:
# define specific annotations to read and exclude the rest.
classnames = [
    'basale', 'bloodvessel', 'collagen', 'fat',
    'granulosum', 'hairfollicle', 'hairroot', 'inner corneum',
    'nerve', 'oil', 'outer corneum', 'roi', 'smoothmuscle',
    'spinosum', 'sweat', 'white','vesselwall']

leftout = ['noise','inflammation'];

In [29]:
# define which images to convert its annotation to xml
LUTpth = r"\\10.162.80.18\Kyu_Sync\Research\Active\Aging_organized\report\Aging Slide registration sheet_210713.xlsx"
LUT = pd.read_excel(LUTpth)
im2use = LUT[LUT['Qupath annotation']==2]['rename']

<ipython-input-29-1c730ac9a808>:3: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  LUT = pd.read_excel(LUTpth)


In [30]:
qupathimnames = [_.image_name for _ in qp.images]

In [31]:
# find index of a's elements within b in order to narrow down b array indirectly.
a = im2use
b = pd.Series(qupathimnames)
idxx = b[b.isin(a)].index
idxx = list(idxx)

In [32]:
# annsrc = r'\\motherserverdw\Kyu_Sync\Research\Active\Aging_organized\svs\class14_annotation_v2'
s2 = time()
for idx,image in enumerate(qp.images[:]):
    if idx in idxx:
        start=time()
        print(idx,image.image_name)
        coords = []
        names = []
        #read annotation
        s4 = time()
        annotations = image.hierarchy.annotations
        if len(annotations)<1: continue
        for annotation in annotations:
            if not annotation.path_class is None:
                # TO-DO: make exclusion of classes dynamic using the defined array above
                if annotation.path_class.name != "inflammation":
                    if annotation.path_class.name != "noise":
                        poly = annotation.roi
                        if poly.type=='MultiPolygon':
                            # coords.append(np.concatenate(tuple([np.array(x.exterior.coords) for x in poly.geoms])).astype(np.int32))
                            dilation_count = 0
                            while poly.type=='MultiPolygon':
                                poly = poly.buffer(1)
                                if dilation_count>10: break
                                poly = poly.buffer(-1)
                                dilation_count += 1
                            # np.append(coords,np.array(poly.exterior.coords).astype(np.int32))
                            # np.append(names,annotation.path_class.name)
                            if not poly.type=='MultiPolygon':
                                coords.append(np.array(poly.exterior.coords).astype(np.int32))
                                names.append(annotation.path_class.name)
                        if poly.type=='Polygon':
                            # np.append(coords,np.array(poly.exterior.coords).astype(np.int32))
                            # np.append(names,annotation.path_class.name)
                            coords.append(np.array(poly.exterior.coords).astype(np.int32))
                            names.append(annotation.path_class.name)
        coords=np.array(coords)
        names =np.array(names)
        print('qupath annotation loaded in {:.2f} sec'.format(time() - s4))
        print(np.unique(names),len(np.unique(names)))
    
        if not set(np.unique(names)).issubset(set(classnames)):
            raise ValueError('check spelling for classes in either imagescope or qupath')
    
        s5=time()
        root = ET.Element("Annotations", MicronsPerPixel="0.494") # for 20X
        # root = ET.Element("Annotations", MicronsPerPixel="0.230500") # for 40X
    
        for idx,elem in enumerate(classnames):
            #basic setup
            m1 = ET.Element("Annotation", Id=str(idx+1), Name="", ReadOnly="0",
                            NameReadOnly="0", LineColorReadOnly="0",
                            Incremental="0", Type="4", LineColor="16711935",
                            Visible="1", Selected="0", MarkupImagePath="", MacroName="")
            root.append(m1)
            b1 = ET.SubElement(m1, "Attributes")
            c1 = ET.SubElement(b1, "Attribute", Name=elem, Id="0", Value="")
            b2 = ET.SubElement(m1, "Regions")
            c2 = ET.SubElement(b2, "RegionAttributeHeaders")
            ET.SubElement(c2, "AttributeHeader", Id="9999", Name="Region", ColumnWidth="-1")
            ET.SubElement(c2, "AttributeHeader", Id="9997", Name="Length", ColumnWidth="-1")
            ET.SubElement(c2, "AttributeHeader", Id="9996", Name="Area", ColumnWidth="-1")
            ET.SubElement(c2, "AttributeHeader", Id="9998", Name="Text", ColumnWidth="-1")
            ET.SubElement(c2, "AttributeHeader", Id="1", Name="Description", ColumnWidth="-1")
            # individual annotation circle
            for annidx,annelem in enumerate(coords[names==elem]):
                d2 = ET.SubElement(b2, "Region", Id=str(annidx+1), Type="0", Zoom="1", Selected="0", ImageLocation="", ImageFocus="-1",
                Text="", NegativeROA="0", InputRegionId="0", Analyze="1",DisplayId=str(annidx+1))
                ET.SubElement(d2,"Attributes")
                e2 = ET.SubElement(d2,"Vertices")
                for vertidx,vertelem in enumerate(annelem):
                    ET.SubElement(e2,"Vertex", X=str(vertelem[0]), Y=str(vertelem[1]), Z="0")
            b3 = ET.SubElement(m1, "Plots")
    
        # xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="   ")
    
        print('combined xml generated in {:.2f} sec'.format(time() - s5))
        print(image.image_name,'time elapsed {:.2f} sec'.format(time() - start))
    
        # with open(image.image_name.replace('svs','xml'),'wb') as ff:
        #     ff.write(xmlstr.encode('utf-8'))
        #
        dstnm = image.image_name.replace('svs','xml')
        dst2 = os.path.join(dstpth,dstnm)
        with open(dst2,'wb') as f:
            ET.ElementTree(root).write(f)
        print(image.image_name.replace('svs','xml'),' file saved')
print(len(qp.images),'files processed in {:.2f} sec'.format(time() - s2))

1 wf21_b4_01.svs
qupath annotation loaded in 0.74 sec
['basale' 'bloodvessel' 'collagen' 'granulosum' 'inner corneum'
 'outer corneum' 'roi' 'spinosum' 'vesselwall' 'white'] 10
combined xml generated in 0.14 sec
wf21_b4_01.svs time elapsed 0.93 sec
wf21_b4_01.xml  file saved
15 wf45_b4_01.svs
qupath annotation loaded in 0.65 sec
['bloodvessel' 'collagen' 'fat' 'nerve' 'roi' 'smoothmuscle' 'sweat'
 'vesselwall' 'white'] 9
combined xml generated in 0.02 sec
wf45_b4_01.svs time elapsed 0.67 sec
wf45_b4_01.xml  file saved
18 wf63_b4_01.svs
qupath annotation loaded in 1.19 sec
['basale' 'bloodvessel' 'collagen' 'granulosum' 'outer corneum' 'roi'
 'spinosum' 'vesselwall' 'white'] 9
combined xml generated in 0.02 sec
wf63_b4_01.svs time elapsed 1.21 sec
wf63_b4_01.xml  file saved
29 wf87_b3_03.svs
qupath annotation loaded in 0.64 sec
['basale' 'bloodvessel' 'collagen' 'granulosum' 'inner corneum'
 'outer corneum' 'roi' 'spinosum' 'vesselwall' 'white'] 10
combined xml generated in 0.02 sec
wf8

<ipython-input-32-29c48ea04b5b>:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  coords=np.array(coords)
